In [ ]:
'''
As mentioned in the writeup i am using code from 
https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/
to setup the spark environment as i was getting errors while running PySpark in my local environment
'''

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://www.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
!pip install faker 

In [ ]:
import findspark
findspark.init()

In [ ]:
import numpy as np
from faker import Faker 
import pandas as pd
import random as rnd
from random import randint
import datetime

fake = Faker()

lsit_of_rows=[]

Airport_list=['BWI','IAD','MIA','LAX','PHL','JFK','TXD','VIR','ATL','LAS','DFW']

for flights in range(0,100):
  lsit_of_rows.append({'FlightNo': ('UM -'+str(randint(100,999))),
                       'Date':fake.date_between(start_date='-2y', end_date='today'),
                       'Time':fake.time(pattern='%H:%M:%S',end_datetime=None),
                       'Actual_date':fake.date_between(start_date='-2y', end_date='today'),
                       'Actual_time':fake.time(pattern='%H:%M:%S',end_datetime=None),
                       'Origin': rnd.choice(Airport_list),
                       'Destination':rnd.choice(Airport_list)})

In [ ]:
#print(lsit_of_rows)
flight_data=pd.DataFrame(lsit_of_rows)
flight_data.head()

,FlightNo,Date,Time,Actual_date,Actual_time,Origin,Destination
0,UM -431,2020-08-15,09:04:44,2021-08-12,16:42:24,TXD,DFW
1,UM -983,2020-05-27,18:42:41,2020-10-17,12:45:55,JFK,LAX
2,UM -139,2020-04-11,06:16:51,2019-12-18,23:08:19,MIA,IAD
3,UM -985,2020-02-06,22:14:18,2020-02-25,18:19:24,LAS,VIR
4,UM -785,2021-03-01,07:50:17,2020-03-31,07:44:09,TXD,MIA


In [ ]:
flight_data['Date']=flight_data['Actual_date']
flight_data.head()

,FlightNo,Date,Time,Actual_date,Actual_time,Origin,Destination
0,UM -431,2021-08-12,09:04:44,2021-08-12,16:42:24,TXD,DFW
1,UM -983,2020-10-17,18:42:41,2020-10-17,12:45:55,JFK,LAX
2,UM -139,2019-12-18,06:16:51,2019-12-18,23:08:19,MIA,IAD
3,UM -985,2020-02-25,22:14:18,2020-02-25,18:19:24,LAS,VIR
4,UM -785,2020-03-31,07:50:17,2020-03-31,07:44:09,TXD,MIA


In [ ]:
flight_data['Scheduled_date_time']=pd.to_datetime(flight_data['Actual_date'].astype(str)+' '+flight_data['Actual_time'])
flight_data['Arrival_date_time']=pd.to_datetime(flight_data['Date'].astype(str)+' '+flight_data['Time'])
flight_data.head()

,FlightNo,Date,Time,Actual_date,Actual_time,Origin,Destination,Scheduled_date_time,Arrival_date_time
0,UM -431,2021-08-12,09:04:44,2021-08-12,16:42:24,TXD,DFW,2021-08-12 16:42:24,2021-08-12 09:04:44
1,UM -983,2020-10-17,18:42:41,2020-10-17,12:45:55,JFK,LAX,2020-10-17 12:45:55,2020-10-17 18:42:41
2,UM -139,2019-12-18,06:16:51,2019-12-18,23:08:19,MIA,IAD,2019-12-18 23:08:19,2019-12-18 06:16:51
3,UM -985,2020-02-25,22:14:18,2020-02-25,18:19:24,LAS,VIR,2020-02-25 18:19:24,2020-02-25 22:14:18
4,UM -785,2020-03-31,07:50:17,2020-03-31,07:44:09,TXD,MIA,2020-03-31 07:44:09,2020-03-31 07:50:17


In [ ]:
flight_data['Origin']=flight_data['Origin'].astype('category')
flight_data['Destination']=flight_data['Destination'].astype('category')

In [ ]:
flight_data['Delay']=abs(flight_data['Arrival_date_time']-flight_data['Scheduled_date_time'])
flight_data['Delay'] = flight_data['Delay'].astype(int)
flight_data.head()

,FlightNo,Date,Time,Actual_date,Actual_time,Origin,Destination,Scheduled_date_time,Arrival_date_time,Delay
0,UM -431,2021-08-12,09:04:44,2021-08-12,16:42:24,TXD,DFW,2021-08-12 16:42:24,2021-08-12 09:04:44,27460000000000
1,UM -983,2020-10-17,18:42:41,2020-10-17,12:45:55,JFK,LAX,2020-10-17 12:45:55,2020-10-17 18:42:41,21406000000000
2,UM -139,2019-12-18,06:16:51,2019-12-18,23:08:19,MIA,IAD,2019-12-18 23:08:19,2019-12-18 06:16:51,60688000000000
3,UM -985,2020-02-25,22:14:18,2020-02-25,18:19:24,LAS,VIR,2020-02-25 18:19:24,2020-02-25 22:14:18,14094000000000
4,UM -785,2020-03-31,07:50:17,2020-03-31,07:44:09,TXD,MIA,2020-03-31 07:44:09,2020-03-31 07:50:17,368000000000


In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
flight_data['FlightNo']=labelencoder.fit_transform(flight_data['FlightNo'])
flight_data['Origin']=labelencoder.fit_transform(flight_data['Origin'])
flight_data['Destination']=labelencoder.fit_transform(flight_data['Destination'])
flight_data['Delay']=labelencoder.fit_transform(flight_data['Delay'])

In [ ]:
flight_data.head()

,FlightNo,Date,Time,Actual_date,Actual_time,Origin,Destination,Scheduled_date_time,Arrival_date_time,Delay
0,37,2021-08-12,09:04:44,2021-08-12,16:42:24,9,2,2021-08-12 16:42:24,2021-08-12 09:04:44,53
1,88,2020-10-17,18:42:41,2020-10-17,12:45:55,4,6,2020-10-17 12:45:55,2020-10-17 18:42:41,45
2,7,2019-12-18,06:16:51,2019-12-18,23:08:19,7,3,2019-12-18 23:08:19,2019-12-18 06:16:51,94
3,89,2020-02-25,22:14:18,2020-02-25,18:19:24,5,10,2020-02-25 18:19:24,2020-02-25 22:14:18,30
4,70,2020-03-31,07:50:17,2020-03-31,07:44:09,9,7,2020-03-31 07:44:09,2020-03-31 07:50:17,1


In [ ]:
flight_data.to_csv('Flights_data.csv',sep=',')

In [ ]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import udf
from pyspark import SparkContext

# For regression and other possible ML tools (Spark)
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.param import Param, Params
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.stat import Statistics

# Important for managing features  (Spark)
from pyspark.ml.feature import OneHotEncoder, StringIndexer,Bucketizer
from pyspark.ml.feature import VectorAssembler

In [ ]:
sc=SparkContext()
sql=SQLContext(sc)
sqlDataframe=sql.createDataFrame(flight_data)
sqlDataframe.limit(5).toPandas().head()


,FlightNo,Date,Time,Actual_date,Actual_time,Origin,Destination,Scheduled_date_time,Arrival_date_time,Delay
0,37,2021-08-12,09:04:44,2021-08-12,16:42:24,9,2,2021-08-12 16:42:24,2021-08-12 09:04:44,53
1,88,2020-10-17,18:42:41,2020-10-17,12:45:55,4,6,2020-10-17 12:45:55,2020-10-17 18:42:41,45
2,7,2019-12-18,06:16:51,2019-12-18,23:08:19,7,3,2019-12-18 23:08:19,2019-12-18 06:16:51,94
3,89,2020-02-25,22:14:18,2020-02-25,18:19:24,5,10,2020-02-25 18:19:24,2020-02-25 22:14:18,30
4,70,2020-03-31,07:50:17,2020-03-31,07:44:09,9,7,2020-03-31 07:44:09,2020-03-31 07:50:17,1


In [ ]:

from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
sqlDataframe = sqlDataframe.withColumn("Date",col("Date").cast(StringType())) \
    .withColumn("Time",col("Time").cast(StringType())) \
    .withColumn("Actual_date",col("Actual_date").cast(StringType())) \
    .withColumn("Scheduled_date_time",col("Scheduled_date_time").cast(StringType())) \
    .withColumn("Arrival_date_time",col("Scheduled_date_time").cast(StringType()))
sqlDataframe.printSchema()

root
 |-- FlightNo: long (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Actual_date: string (nullable = true)
 |-- Actual_time: string (nullable = true)
 |-- Origin: long (nullable = true)
 |-- Destination: long (nullable = true)
 |-- Scheduled_date_time: string (nullable = true)
 |-- Arrival_date_time: string (nullable = true)
 |-- Delay: long (nullable = true)



In [ ]:
from pyspark.ml.feature import FeatureHasher
hasher = FeatureHasher(inputCols=["Date", "Time", "Actual_date", "Actual_time","FlightNo",'Origin','Destination','Scheduled_date_time','Arrival_date_time'],
                       outputCol="features")

featurized = hasher.transform(sqlDataframe)
featurized.show(truncate=False)

+--------+----------+--------+-----------+-----------+------+-----------+-------------------+-------------------+-----+-----------------------------------------------------------------------------------------------------------------+
|FlightNo|Date      |Time    |Actual_date|Actual_time|Origin|Destination|Scheduled_date_time|Arrival_date_time  |Delay|features                                                                                                         |
+--------+----------+--------+-----------+-----------+------+-----------+-------------------+-------------------+-----+-----------------------------------------------------------------------------------------------------------------+
|37      |2021-08-12|09:04:44|2021-08-12 |16:42:24   |9     |2          |2021-08-12 16:42:24|2021-08-12 16:42:24|53   |(262144,[16561,115717,141353,155946,159630,180358,200995,209246,233293],[1.0,1.0,1.0,1.0,1.0,2.0,1.0,37.0,9.0])  |
|88      |2020-10-17|18:42:41|2020-10-17 |12:45:55   |4     |6  

In [ ]:
splits = featurized.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]
train_df.show()

+--------+----------+--------+-----------+-----------+------+-----------+-------------------+-------------------+-----+--------------------+
|FlightNo|      Date|    Time|Actual_date|Actual_time|Origin|Destination|Scheduled_date_time|  Arrival_date_time|Delay|            features|
+--------+----------+--------+-----------+-----------+------+-----------+-------------------+-------------------+-----+--------------------+
|       0|2021-02-25|23:05:46| 2021-02-25|   10:57:18|     9|          9|2021-02-25 10:57:18|2021-02-25 10:57:18|   77|(262144,[58828,91...|
|       2|2021-09-26|04:47:25| 2021-09-26|   16:39:03|     6|          8|2021-09-26 16:39:03|2021-09-26 16:39:03|   75|(262144,[8428,251...|
|       6|2020-10-24|13:58:03| 2020-10-24|   03:05:41|     0|         10|2020-10-24 03:05:41|2020-10-24 03:05:41|   68|(262144,[66981,85...|
|       7|2019-12-18|06:16:51| 2019-12-18|   23:08:19|     7|          3|2019-12-18 23:08:19|2019-12-18 23:08:19|   94|(262144,[96341,13...|
|      13|202

In [ ]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='Delay', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: (262144,[542,1485,2564,3773,4834,5859,6690,6923,8428,9106,9220,12123,12717,14078,16038,16629,17009,17082,17087,18291,18697,18750,19631,19712,20882,22201,23081,23098,23152,23910,24026,25117,26574,27188,27579,30300,30794,31255,31362,31993,32714,33687,35556,35812,36040,36790,37194,37925,38102,39468,42472,43795,44264,44457,44461,44557,46096,46661,47050,47233,47308,47639,48497,49329,49337,53472,54070,58096,58164,58828,60739,63484,63797,63816,64074,64247,65130,65258,65710,66157,66377,66981,68013,69318,69444,69621,70761,73112,73836,74190,75633,75837,77664,78571,80016,80688,81561,82467,84551,85120,85614,85620,86036,87103,87120,87429,89072,89144,89583,89958,91115,91207,91537,91618,92321,96341,98664,98762,100496,101205,103414,104075,104151,104797,105617,106441,106695,106935,107232,107491,108378,109942,110234,111285,112945,113699,113971,115530,116866,117213,118312,118954,119592,119736,120271,121472,121679,124849,124880,126532,126788,126978,127328,127697,129032,130504,132164,132309,1

In [ ]:
predictions = lr_model.transform(test_df)
predictions.select("prediction","Delay","features").show()

+------------------+-----+--------------------+
|        prediction|Delay|            features|
+------------------+-----+--------------------+
| 58.14332512283955|   22|(262144,[3672,796...|
| 57.77870443227833|   21|(262144,[28758,15...|
|57.414083741717114|   78|(262144,[18006,80...|
| 57.23177339643651|   38|(262144,[35847,81...|
| 56.32022167003347|   67|(262144,[14648,20...|
| 54.13249752666618|   53|(262144,[16561,11...|
|53.585566490824355|   74|(262144,[3744,822...|
| 53.40325614554375|   60|(262144,[52991,12...|
| 51.94477338329889|   17|(262144,[34395,11...|
| 50.85091131161524|   36|(262144,[33081,52...|
|50.121669930492814|    0|(262144,[32511,10...|
| 49.57473889465099|   52|(262144,[87228,11...|
| 48.48087682296734|   87|(262144,[72241,13...|
|  47.5693250965643|   19|(262144,[8365,358...|
| 46.47546302488065|   11|(262144,[8540,111...|
| 44.65235957207458|   30|(262144,[20025,31...|
|44.470049226793975|   55|(262144,[56180,62...|
|60.513359611487445|    5|(262144,[28319